In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
import importlib
import mr_rrn, data_aux
from tensorflow.contrib import learn

In [ ]:
# Settings test
examples=50000
num_seq = 4
batch_size = 200

learning_rate = 0.05
gradient_clipping = 1.

n_hidden_coarse_prediction = 2
coarse_kwargs = {'embedding_shape' : [4,4],
                 'num_seq' : num_seq,
                 'num_steps' : 10,
                 'batch_size' : batch_size,
                 'n_hidden_encoder' : 10,
                 'n_hidden_context' : 9,
                 'n_hidden_decoder' : 8}
nl_kwargs = {'embedding_shape' : [4,4],
             'num_seq' : num_seq,
             'num_steps' : 20,
             'batch_size' : batch_size,
             'n_hidden_encoder' : 9,
             'n_hidden_context' : 8, 
             'n_hidden_decoder' : 10}

In [ ]:
# Load data
importlib.reload(mr_rrn)
importlib.reload(data_aux)

# Simple test
coarse_data, coarse_length, nl_data, nl_length = data_aux.generate_test_data_with_coarse(examples=examples, num_seq=num_seq, num_steps_nl=nl_kwargs['num_steps'], num_steps_coarse=coarse_kwargs['num_steps'])
coarse_W_embedding = np.eye(4)
nl_W_embedding = np.eye(4)

batches = data_aux.batch_iter([coarse_data, coarse_length, nl_data, nl_length],batch_size=batch_size, num_epochs=5)

# Build graph
timer = datetime.now()
tf.reset_default_graph()
graph_nodes = mr_rrn.build_graph(coarse_kwargs, n_hidden_coarse_prediction, nl_kwargs, learning_rate, gradient_clipping)
timer = datetime.now() - timer
print('Time build graph: ', timer)

# Train
acc_loss = 0
with tf.Session() as sess:
    timer = datetime.now()
    feed_dict={graph_nodes['coarse_vocab_input'] : coarse_W_embedding,
               graph_nodes['nl_vocab_input'] : nl_W_embedding
              }
    sess.run(tf.initialize_all_variables(), feed_dict=feed_dict)
    timer = datetime.now() - timer
    print('Time init graph: ', timer)
    timer = datetime.now()
    acc_loss = 0
    for i, batch in enumerate(batches):
        coarse_seq, coarse_len, nl_seq, nl_len = batch
        feed_dict = {graph_nodes['coarse_sequence_input'] : coarse_seq,
                     graph_nodes['coarse_length_input'] : coarse_len,
                     graph_nodes['nl_sequence_input'] : nl_seq,
                     graph_nodes['nl_length_input'] : nl_len
                     }
        loss, _ = sess.run([graph_nodes['total_loss'], graph_nodes['train_step']], feed_dict=feed_dict)
        acc_loss += loss
        if i % 25 == 0 and i > 0:
            print(acc_loss/25)
            acc_loss = 0
timer = datetime.now() - timer
print('Time run graph: ', timer)